# Use ONNX model converted from LightGBM

This notebook facilitates ONNX, LightGBM, and watsonx.ai Runtime service. It contains steps and code to work with [ibm-watsonx-ai](https://pypi.python.org/pypi/ibm-watsonx-ai) library available in PyPI repository in order to convert the model to ONNX format. It also introduces commands for getting model and training data, persisting model, deploying model, and scoring it.

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Learning goals

The learning goals of this notebook are:

-  Train a LightGBM model
-  Convert the LightGBM model to ONNX format
-  Persist the converted model in the watsonx.ai Runtime repository
-  Deploy the model for online scoring using client library
-  Score sample records using the client library


## Contents

This notebook contains the following parts:

1.	[Setting up the environment](#setup)
2.	[Training a LightGBM model](#download)
3.	[Converting the LightGBM model to ONNX format](#convert)
4.	[Persisting the converted ONNX model](#upload)
5.	[Deploying and scoring the ONNX model](#deploy)
6.  [Cleanup](#cleanup)
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setting up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the `ibm-watsonx-ai` and dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
!pip install -U ibm-watsonx-ai | tail -n 1
!pip install scikit-learn==1.3.0 | tail -n 1
!pip install lightgbm==4.2.0 | tail -n 1
!pip install onnxruntime==1.16.3 | tail -n 1
!pip install onnx==1.16.2 | tail -n 1

In [2]:
import json
import onnxruntime as ort
from ibm_watsonx_ai import Credentials, APIClient

### Connection to watsonx.ai Runtime

Authenticate with the watsonx.ai Runtime service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

Get the value of `api_key` from the output.


Location of your watsonx.ai Runtime instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance watsonx.ai_RUNTIME_INSTANCE_NAME
```

Get the value of `location` from the output.

**Tip**: You can generate your `Cloud API key` by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service-specific url by going to the [**Endpoint URLs** section of the watsonx.ai Runtime docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance details.

You can also get the service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key, and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [3]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'us-south'

In [4]:
credentials = Credentials(
    api_key=api_key,
    url=f'https://{location}.ml.cloud.ibm.com'
)

In [5]:
client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have a space, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select watsonx.ai Runtime instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [8]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use the `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in watsonx.ai Runtime, you need to set **space** which you will be using.

In [9]:
client.set.default_space(space_id)

'SUCCESS'

<a id="download"></a>
## 2. Training a LightGBM model
In this section, you will train a sample LightGBM model.

### 2.1. Train the model
Load the iris data first.

In [34]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd

iris = datasets.load_iris()
x, y = iris.data, iris.target
x_df = pd.DataFrame(x, columns= iris.feature_names)
x_train, x_test, y_train, y_test = train_test_split(x_df, y, test_size=0.15)

Train the model.

In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(objective='multiclass')
model.fit(x_train, y_train);

### 2.2 Test the model (`predict`)

In [66]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_test)
cr = classification_report(y_test, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         7
           2       1.00      1.00      1.00         6

    accuracy                           1.00        23
   macro avg       1.00      1.00      1.00        23
weighted avg       1.00      1.00      1.00        23



<a id="convert"></a>
## 3. Converting the LightGBM model to ONNX format
In this section, you will convert the model to the ONNX format.

### 3.1 Converting the model

In [67]:
from onnxmltools import convert_lightgbm
from skl2onnx.common.data_types import FloatTensorType
from onnxmltools.utils import save_model

initial_types = [("float_input", FloatTensorType([None, model.n_features_]))]
onnx_model = convert_lightgbm(model=model, initial_types=initial_types, zipmap=False)
onnx_model_filename = "lgbm_model.onnx"
save_model(onnx_model, onnx_model_filename)

The maximum opset needed by this model is only 9.


### 3.3. Validating the ONNX Model

After exporting the model, you should verify its integrity and ensure that it functions as expected. We will use `onnxruntime` to load the model and perform inference on the random dataset. Additionally, we’ll use `onnx`'s `checker` module to validate the exported ONNX model.


In [68]:
import onnx
onnx.checker.check_model(onnx_model)

In [69]:
eval_data = [[4, 3, 1, 0],
             [5, 2, 3, 6]]

In [70]:
session = ort.InferenceSession(onnx_model_filename)

input_data = {session.get_inputs()[0].name: eval_data}
session.run([], input_data)

2025-01-09 18:04:30.385896 [W:onnxruntime:, execution_frame.cc:857 VerifyOutputSizes] Expected shape from model of {1} does not match actual shape of {2} for output label


[array([0, 2], dtype=int64),
 array([[9.9993360e-01, 6.4027365e-05, 2.3447901e-06],
        [8.5499021e-04, 1.2945167e-02, 9.8619986e-01]], dtype=float32)]

<a id="upload"></a>
## 4. Persisting the converted ONNX model

In this section, you will learn how to store your converted ONNX model in watsonx.ai Runtime.

### 4.1. Publishing the model in watsonx.ai Runtime repository

Define model name, type and software spec.

In [71]:
sofware_spec_id = client.software_specifications.get_id_by_name("onnxruntime_opset_19")
onnx_model_zip = "lgbm_onnx.zip"

In [75]:
!zip {onnx_model_zip} {onnx_model_filename}

  adding: lgbm_model.onnx (deflated 88%)


In [76]:
metadata = {
    client.repository.ModelMetaNames.NAME: 'LGBM to ONNX converted model',
    client.repository.ModelMetaNames.TYPE: 'onnxruntime_1.16',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_ID: sofware_spec_id
}

published_model = client.repository.store_model(
    model=onnx_model_zip,
    meta_props=metadata
)

### 4.2. Get model details

In [77]:
published_model_id = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_id)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "hybrid_pipeline_software_specs": [],
    "software_spec": {
      "id": "368d2795-aaa7-59a0-834c-248c64a5a99e",
      "name": "onnxruntime_opset_19"
    },
    "type": "onnxruntime_1.16"
  },
  "metadata": {
    "created_at": "2025-01-09T17:06:11.310Z",
    "id": "8099409e-b892-48e7-8b6a-1ae5f438d722",
    "modified_at": "2025-01-09T17:06:16.738Z",
    "name": "LGBM to ONNX converted model",
    "owner": "IBMid-270002BE4G",
    "resource_key": "d9645a23-bdb0-4e23-85df-7290765fc2fd",
    "space_id": "f85a48fb-3982-4b60-900f-ce156e844be4"
  },
  "system": {
    "warnings": []
  }
}


<a id="deploy"></a>
## 5. Deploying and scoring the ONNX model

In this section you'll learn how to create an online scoring service and predict on unseen data.

### 5.1. Create online deployment for published model

In [78]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of LGBM to ONNX converted model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_id, meta_props=metadata)



######################################################################################

Synchronous deployment creation for id: '8099409e-b892-48e7-8b6a-1ae5f438d722' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
..
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='8693408a-2ba5-4eeb-8f55-ce4088521213'
-----------------------------------------------------------------------------------------------




In [79]:
deployment_id = client.deployments.get_id(created_deployment)

Now you can print an online scoring endpoint. 

In [80]:
client.deployments.get_scoring_href(created_deployment)

'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/8693408a-2ba5-4eeb-8f55-ce4088521213/predictions'

### 5.2. Get deployment details

In [81]:
client.deployments.get_details(deployment_id)

Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.


{'entity': {'asset': {'id': '8099409e-b892-48e7-8b6a-1ae5f438d722'},
  'custom': {},
  'deployed_asset_type': 'model',
  'hardware_spec': {'id': 'e7ed1d6c-2e89-42d7-aed5-863b972c1d2b',
   'name': 'S',
   'num_nodes': 1},
  'name': 'Deployment of LGBM to ONNX converted model',
  'online': {},
  'space_id': 'f85a48fb-3982-4b60-900f-ce156e844be4',
  'status': {'inference': [{'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/8693408a-2ba5-4eeb-8f55-ce4088521213/predictions'}],
   'online_url': {'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/8693408a-2ba5-4eeb-8f55-ce4088521213/predictions'},
   'serving_urls': ['https://us-south.ml.cloud.ibm.com/ml/v4/deployments/8693408a-2ba5-4eeb-8f55-ce4088521213/predictions'],
   'state': 'ready'}},
 'metadata': {'created_at': '2025-01-09T17:06:25.490Z',
  'id': '8693408a-2ba5-4eeb-8f55-ce4088521213',
  'modified_at': '2025-01-09T17:06:25.490Z',
  'name': 'Deployment of LGBM to ONNX converted model',
  'owner': 'IBMid-270002BE4G'

<a id="score"></a>
### 5.3. Score

You can use the method shown below to perform a test scoring request against the deployed model.

Prepare scoring payload with records to score.

In [82]:
scoring_payload = {
    "input_data": [
        {
            "values": eval_data

        }
    ]
}

Use ``client.deployments.score()`` method to run scoring.

In [83]:
predictions = client.deployments.score(deployment_id, scoring_payload)

Let's print the result of predictions.

In [84]:
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "id": "label",
      "values": [
        0,
        2
      ]
    },
    {
      "id": "probabilities",
      "values": [
        [
          0.9999336004257202,
          6.402736471500248e-05,
          2.344790118513629e-06
        ],
        [
          0.0008549902122467756,
          0.012945166788995266,
          0.9861998558044434
        ]
      ]
    }
  ]
}


As you can see, predicted values are the same as the ones that were based on the test dataset.

<a id="cleanup"></a>
## 6. Cleanup

If you want to clean up after the notebook execution, i.e. remove any created assets like:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 7. Summary and next steps

 You successfully completed this notebook! You learned how to use ONNX, TensorFlow machine learning library as well as watsonx.ai Runtime for model creation and deployment. Check out our _[Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html?context=wx)_ for more samples, tutorials, documentation, how-tos, and blog posts.

Copyright © 2025 IBM. This notebook and its source code are released under the terms of the MIT License.